In [ ]:
import time
import paho.mqtt.client as mqtt

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe("no_1/temperatura")
    client.subscribe("no_1/umidade")

# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    print(msg.topic+" "+str(msg.payload))
    

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.username_pw_set("ha", "jdep260")
client.connect("192.168.43.81", 1883, 60)

# Blocking call that processes network traffic, dispatches callbacks and
# handles reconnecting.
# Other loop*() functions are available that give a threaded interface and a
# manual interface.
# client.subscribe("teste",0)


print("Publishing message to topic","house/bulbs/bulb1")
# client.publish("teste","OFF")
time.sleep(5)
client.publish("no_1/temperatura","hmmmmmm")

client.loop_forever()

# topic valefruit/farm01/esp01/sensors/air_humidity
# topic valefruit/farm01/esp01/sensors/air_temperature
# topic valefruit/farm01/esp01/sensors/light
# topic valefruit/farm01/esp01/sensors/soil_humidity


# topic valefruit/farm01/no_01/sensores/air_humidity
# topic valefruit/farm01/no_01/sensores/air_temperature
# topic valefruit/farm01/no_01/sensores/light
# topic valefruit/farm01/no_01/sensores/soil_humidity


In [ ]:
import time
import paho.mqtt.client as paho
# broker="broker.hivemq.com"
broker="10.10.1.55"
#define callback
def on_message(client, userdata, message):
    time.sleep(1)
    print("received message =",str(message.payload.decode("utf-8")))

client= paho.Client("client-001") #create client object client1.on_publish = on_publish #assign function to callback client1.connect(broker,port) #establish connection client1.publish("house/bulb1","on")
######Bind function to callback
client.on_message=on_message
#####
print("connecting to broker ",broker)
client.connect(broker)#connect
client.loop_start() #start loop to process received messages
print("subscribing ")
client.subscribe("teste")#subscribe
time.sleep(2)
print("publishing ")
client.publish("house/bulb1","on")#publish
time.sleep(4)
client.disconnect() #disconnect
client.loop_stop() #stop loop

In [11]:
# Exemplo 01 - 68 uvas no cacho
# Exemplo 02 - 58 uvas no cacho
# Uvas cilíndricas

# Standard imports
from __future__ import print_function
import cv2 as cv2
import numpy as np
import argparse
import random as rng

 
# Read image
# im = cv2.imread("blob.jpg", cv2.IMREAD_GRAYSCALE)
# im = cv2.imread("blob.jpg")

im = cv2.imread("img/blob2.png")

# cv2.imshow("Keypoints", im)
# cv2.waitKey() #image will not show until this is called
# cv2.destroyWindow("Keypoints",) #make sure window closes cleanly
# Set up the detector with default parameters.
detector = cv2.SimpleBlobDetector_create()
 
# Detect blobs.
keypoints = detector.detect(im)
 
# Draw detected blobs as red circles.
# cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
 
# Show keypoints
# cv2.imshow("Keypoints", im_with_keypoints)
# cv2.waitKey(0)


threshold = 100

src = im
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

img = cv2.imread("img/blob2.png",0)
img = cv2.medianBlur(img,5)
cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)


src_gray = cv2.blur(src_gray, (3,3))

canny_output2 = cv2.Canny(src_gray, 180, 200, 3)  
blur = cv2.GaussianBlur(canny_output2, (5,5), 0)

circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,20,
                            param1=50,param2=30,minRadius=10,maxRadius=0)

circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    # draw the outer circle
    cv2.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle
    cv2.circle(cimg,(i[0],i[1]),2,(0,0,255),3)




canny_output = cv2.Canny(src_gray, threshold, threshold * 2)



_, contours, _ = cv2.findContours(canny_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)


contours_poly = [None]*len(contours)
boundRect = [None]*len(contours)
centers = [None]*len(contours)
radius = [None]*len(contours)
for i, c in enumerate(contours):
    contours_poly[i] = cv2.approxPolyDP(c, 3, True)
    boundRect[i] = cv2.boundingRect(contours_poly[i])
    centers[i], radius[i] = cv2.minEnclosingCircle(contours_poly[i])


drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)


for i in range(len(contours)):
    color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
    cv2.drawContours(drawing, contours_poly, i, color)
#     cv2.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), \
#       (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
    cv2.circle(drawing, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)



# cv2.imshow('Contours', circles)
cv2.imshow('detected circles',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
from __future__ import print_function
import cv2 as cv
import numpy as np
import argparse
import random as rng
rng.seed(12345)
def thresh_callback(val):
    threshold = val
    
    canny_output = cv.Canny(src_gray, threshold, threshold * 2)
    
    
    _, contours, _ = cv.findContours(canny_output, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    
    
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    centers = [None]*len(contours)
    radius = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv.approxPolyDP(c, 3, True)
        boundRect[i] = cv.boundingRect(contours_poly[i])
        centers[i], radius[i] = cv.minEnclosingCircle(contours_poly[i])
    
    
    drawing = np.zeros((canny_output.shape[0], canny_output.shape[1], 3), dtype=np.uint8)
    
    
    for i in range(len(contours)):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        cv.drawContours(drawing, contours_poly, i, color)
        cv.rectangle(drawing, (int(boundRect[i][0]), int(boundRect[i][1])), \
          (int(boundRect[i][0]+boundRect[i][2]), int(boundRect[i][1]+boundRect[i][3])), color, 2)
        cv.circle(drawing, (int(centers[i][0]), int(centers[i][1])), int(radius[i]), color, 2)
    
    
    cv.imshow('Contours', drawing)
    
parser = argparse.ArgumentParser(description='Code for Creating Bounding boxes and circles for contours tutorial.')
parser.add_argument('--input', help='Path to input image.', default='stuff.jpg')
args = parser.parse_args()
src = cv.imread(cv.samples.findFile(args.input))
if src is None:
    print('Could not open or find the image:', args.input)
    exit(0)
# Convert image to gray and blur it
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
src_gray = cv.blur(src_gray, (3,3))
source_window = 'Source'
cv.namedWindow(source_window)
cv.imshow(source_window, src)
max_thresh = 255
thresh = 100 # initial threshold
cv.createTrackbar('Canny thresh:', source_window, thresh, max_thresh, thresh_callback)
thresh_callback(thresh)
cv.waitKey()

usage: ipykernel_launcher.py [-h] [--input INPUT]
ipykernel_launcher.py: error: unrecognized arguments: -f /run/user/1000/jupyter/kernel-dbc53bf8-e0c3-44ce-b089-3db6c5f7ec9d.json


SystemExit: 2

/home/ruan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# WATERSHED

In [ ]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
img = cv.imread('blob.jpg')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)


# noise removal

# noise removal
kernel = np.ones((3,3),np.uint8)
opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
# sure background area
sure_bg = cv.dilate(opening,kernel,iterations=3)
# Finding sure foreground area
dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
# Finding unknown region
sure_fg = np.uint8(sure_fg)
unknown = cv.subtract(sure_bg,sure_fg)


# Marker labelling
ret, markers = cv.connectedComponents(sure_fg)
# Add one to all labels so that sure background is not 0, but 1
markers = markers+1
# Now, mark the region of unknown with zero
markers[unknown==255] = 0

markers = cv.watershed(img,markers)
img[markers == -1] = [255,0,0]

cv.imshow('Source',img)
cv.waitKey(0)
cv.destroyAllWindows()

In [12]:
import cv2

raw_image = cv2.imread('img/blob.jpg')
cv2.imshow('Original Image', raw_image)
cv2.waitKey(0)

bilateral_filtered_image = cv2.bilateralFilter(raw_image, 5, 175, 175)
cv2.imshow('Bilateral', bilateral_filtered_image)
cv2.waitKey(0)


edge_detected_image = cv2.Canny(bilateral_filtered_image, 75, 200)
cv2.imshow('Edge', edge_detected_image)
cv2.waitKey(0)

_, contours, hierarchy = cv2.findContours(edge_detected_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

contour_list = []
for contour in contours:
    approx = cv2.approxPolyDP(contour,0.01*cv2.arcLength(contour,True),True)
    area = cv2.contourArea(contour)
    if ((len(approx) > 8) & (len(approx) < 23) & (area > 30) ):
        contour_list.append(contour)

cv2.drawContours(raw_image, contour_list,  -1, (255,0,0), 2)
cv2.imshow('Objects Detected',raw_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# USAGE
# python watershed.py --image images/coins_01.png

# import the necessary packages
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import numpy as np
import argparse
import cv2

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
#     help="path to input image")
# args = vars(ap.parse_args())

# load the image and perform pyramid mean shift filtering
# to aid the thresholding step
# image = cv2.imread(args["image"])
image = cv2.imread("img/blob10.jpg")
image = 255-image
shifted = cv2.pyrMeanShiftFiltering(image, 21, 51)
cv2.imshow("Input", image)
cv2.imshow("Shifted", shifted)

# convert the mean shift image to grayscale, then apply
# Otsu's thresholding
gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255,
    cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
print("NON-ZERO THRESH: ", cv2.countNonZero(thresh))
cv2.imshow("Thresh", thresh)

# compute the exact Euclidean distance from every binary
# pixel to the nearest zero pixel, then find peaks in this
# distance map
D = ndimage.distance_transform_edt(thresh)
localMax = peak_local_max(D, indices=False, min_distance=5,
    labels=thresh)

# perform a connected component analysis on the local peaks,
# using 8-connectivity, then appy the Watershed algorithm
markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
labels = watershed(-D, markers, mask=thresh)
print("[INFO] {} unique segments found".format(len(np.unique(labels)) - 1))

# loop over the unique labels returned by the Watershed
# algorithm 
cont = 0
for label in np.unique(labels):
    # if the label is zero, we are examining the 'background'
    # so simply ignore it
    if label == 0:
        continue

    # otherwise, allocate memory for the label region and draw
    # it on the mask
    mask = np.zeros(gray.shape, dtype="uint8")
    mask[labels == label] = 255

    # detect contours in the mask and grab the largest one
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)[-2]
    c = max(cnts, key=cv2.contourArea)
    # draw a circle enclosing the object
    ((x, y), r) = cv2.minEnclosingCircle(c)
    
    print("#{}".format(label), r)
    if r > 14:
        cont+=1
        cv2.circle(image, (int(x), int(y)), int(r), (0, 255, 0), 2)
#     cv2.putText(image, "#{}".format(label), (int(x) - 10, int(y)),
#         cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
print("Contagem: ", cont)
# show the output image
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
# 21
# 63

NON-ZERO THRESH:  304204
[INFO] 67 unique segments found
#1 81.0711669921875
#2 102.62619018554688
#3 41.967350006103516
#4 20.254343032836914
#5 43.33562088012695
#6 46.33687210083008
#7 22.696109771728516
#8 5.005274772644043
#9 100.83943176269531
#10 91.16067504882812
#11 25.7441463470459
#12 21.494489669799805
#13 12.128867149353027
#14 5.4084272384643555
#15 38.347198486328125
#16 22.203702926635742
#17 28.50469970703125
#18 22.638561248779297
#19 162.5657196044922
#20 80.12649536132812
#21 3.5356338024139404
#22 57.030792236328125
#23 30.93670082092285
#24 52.418846130371094
#25 22.0427303314209
#26 41.48446273803711
#27 64.01084899902344
#28 53.57442092895508
#29 6.96429443359375
#30 14.916534423828125
#31 11.011457443237305
#32 60.2369499206543
#33 79.83944702148438
#34 12.661468505859375
#35 21.00605010986328
#36 11.011457443237305
#37 62.81729507446289
#38 14.322646141052246
#39 29.244001388549805
#40 24.186872482299805
#41 64.55262756347656
#42 31.53579330444336
#43 28.53954